In [23]:
import pandas as pd

In [24]:
cr = pd.read_csv("CreditRisk.csv")

In [25]:
cr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            981 non-null    object 
 1   Gender             957 non-null    object 
 2   Married            978 non-null    object 
 3   Dependents         956 non-null    float64
 4   Education          981 non-null    object 
 5   Self_Employed      926 non-null    object 
 6   ApplicantIncome    981 non-null    int64  
 7   CoapplicantIncome  981 non-null    float64
 8   LoanAmount         954 non-null    float64
 9   Loan_Amount_Term   961 non-null    float64
 10  Credit_History     902 non-null    float64
 11  Property_Area      981 non-null    object 
 12  Loan_Status        981 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 99.8+ KB


In [26]:
cr.isnull().sum()[cr.isnull().sum()>0]

Gender              24
Married              3
Dependents          25
Self_Employed       55
LoanAmount          27
Loan_Amount_Term    20
Credit_History      79
dtype: int64

In [27]:
cr.Credit_History.value_counts()

1.0    754
0.0    148
Name: Credit_History, dtype: int64

In [28]:
cr = cr.drop("Loan_ID", axis = 1)

In [29]:
cr = cr.dropna()

In [30]:
# cr.Gender.fillna("Male", inplace = True)
# cr.Married.fillna("Yes", inplace = True)
# cr.Dependents.fillna(0, inplace = True)
# cr.Self_Employed.fillna("No", inplace = True)
# cr.LoanAmount.fillna(cr.LoanAmount.median(), inplace = True)
# cr.Loan_Amount_Term.fillna(360, inplace = True)
# cr.Credit_History.fillna(1, inplace = True)

In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [32]:
cr[cr.select_dtypes(include = "object").columns] = cr[cr.select_dtypes(include = "object").columns].apply(le.fit_transform)

In [33]:
cr_x = cr.iloc[:, 0:-1]
cr_y = cr.iloc[:, -1]

In [34]:
import numpy as np
cr_x = np.array(cr_x)
cr_y = np.array(cr_y)

In [35]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [36]:
from boruta import BorutaPy

In [37]:
feature_selector = BorutaPy(rfc, max_iter = 30, verbose = 2)

In [38]:
feature_selector.fit(cr_x, cr_y)

Iteration: 	1 / 30
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	2 / 30
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	3 / 30
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	4 / 30
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	5 / 30
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	6 / 30
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	7 / 30
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	8 / 30
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	9 / 30
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	10 / 30
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	11 / 30
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	12 / 30
Confirmed: 	3
Tentative: 	0
Rejected: 	8


BorutaPy finished running.

Iteration: 	13 / 30
Confirmed: 	3
Tentative: 	0
Rejected: 	8


BorutaPy(estimator=RandomForestClassifier(n_estimators=1000,
                                          random_state=RandomState(MT19937) at 0x1FA7FEBC040),
         max_iter=30, random_state=RandomState(MT19937) at 0x1FA7FEBC040,
         verbose=2)

In [39]:
feature_selector.support_

array([False, False, False, False, False,  True, False,  True, False,
        True, False])

In [40]:
df = pd.DataFrame()

In [41]:
df["features"] = cr.iloc[:,0:-1].columns
df["Imo"] = feature_selector.support_

In [42]:
df[df.Imo == True]

,features,Imo
5,ApplicantIncome,True
7,LoanAmount,True
9,Credit_History,True


In [43]:
cr.Credit_History.fillna(1, inplace = True)
cr.LoanAmount.fillna(cr.LoanAmount.median(), inplace = True)

In [44]:
new_cr = cr[["ApplicantIncome", "LoanAmount", "Credit_History", "Loan_Status"]]

In [45]:
new_cr[new_cr.select_dtypes(include = "object").columns] = new_cr[new_cr.select_dtypes(include = "object").columns].apply(le.fit_transform)

In [46]:
new_cr.head()

,ApplicantIncome,LoanAmount,Credit_History,Loan_Status
1,4583,128.0,1.0,0
2,3000,66.0,1.0,1
3,2583,120.0,1.0,1
4,6000,141.0,1.0,1
5,5417,267.0,1.0,1


In [48]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(new_cr.iloc[:, 0:-1], new_cr.iloc[:, -1], test_size= .2)

In [49]:
print(train_x.shape)
print(train_y.shape)

print(test_x.shape)
print(test_y.shape)

(615, 3)
(615,)
(154, 3)
(154,)


In [56]:
rfc = RandomForestClassifier(class_weight="balanced")

In [57]:
rfc.fit(train_x, train_y)

RandomForestClassifier(class_weight='balanced')

In [58]:
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score

In [59]:
pred = rfc.predict(test_x)

In [60]:
confusion_matrix(test_y, pred)

array([[ 19,  14],
       [ 11, 110]], dtype=int64)

In [61]:
accuracy_score(test_y, pred)

0.8376623376623377

In [62]:
recall_score(test_y, pred)

0.9090909090909091